In [1]:
import numpy as np
from sklearn.datasets import make_blobs

In [3]:
N = 100
X = np.sort(np.random.rand(N), axis=0)
y = 3 * X - 0.5
y = y + 0.2 * np.random.randn(N)

In [5]:
import torch
import torch.nn as nn
X = torch.from_numpy(X).reshape(-1, 1).float()
y = torch.from_numpy(y).float()

In [6]:
class LinearSVR():
    def __init__(self, input_dim, eps=0.1, lambd=1e-4, batch_size=100, lr=0.1, iters=100):
        self.input_dim = input_dim
        self.eps = eps
        self.lambd = lambd
        self.batch_size = batch_size
        self.lr = lr
        self.iters = iters
        self.w = nn.Parameter(torch.randn(input_dim))
        self.b = nn.Parameter(torch.randn(1))
    
    def _loss(self, x, y):
        out = (y - (torch.matmul(x, self.w) + self.b)).abs()
        out = torch.clamp(out-self.eps, min=0)
        loss = 0.5 * self.lambd * torch.norm(self.w)**2 + out.mean()
        return loss
    
    def _zero_grad(self):
        if self.w.grad is not None:
            self.w.grad.zero_()
            self.b.grad.zero_()
    
    def _update_weights(self):
        self.w.data -= self.lr * self.w.grad.data
        self.b.data -= self.lr * self.b.grad.data
    
    def fit(self, data, labels):
        N = len(data)
        for i in range(self.iters):
            idx = np.random.choice(N, size=self.batch_size, replace=False)
            batch = data[idx]
            label = labels[idx]
            self._zero_grad()
            loss = self._loss(batch, label)
            loss.backward()
            self._update_weights()
    
    def predict(self, data):
        with torch.no_grad():
            pred = torch.matmul(data, self.w.data) + self.b.data
        
        return pred

In [12]:
eps = 0.5
X = X.reshape(-1, 1)
clf = LinearSVR(1, eps=eps, iters=500)

In [ ]:
clf.fit(X, y)

In [14]:
pred = clf.predict(X)